In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import os
import numpy as np

In [2]:
os.getcwd()

'/Users/bipinkarki/Desktop/ML-ABKS-Music-Recommender-System/Notebooks'

In [5]:
tracks = pd.read_csv(r"datasets/tracks.csv")

In [16]:
tracks.tail(5)

,track-id,track-name,genre,artist-id,artist-name,rating,popularity,instruments
9995,9996,Track 9996,Classical,313,Artist 313,1.660654,6,"['Trumpet', 'Bass', 'Guitar', 'Drums', 'Cello'..."
9996,9997,Track 9997,Blues,558,Artist 558,2.807437,3,"['Vocals', 'Guitar', 'Saxophone', 'Piano']"
9997,9998,Track 9998,Latin,673,Artist 673,1.575468,9,"['Drums', 'Guitar', 'Saxophone', 'Violin', 'Pi..."
9998,9999,Track 9999,Hip Hop,106,Artist 106,0.490193,2,"['Piano', 'Guitar', 'Drums', 'Violin', 'Trumpe..."
9999,10000,Track 10000,Classical,423,Artist 423,4.105675,0,"['Guitar', 'Drums', 'Piano', 'Bass']"


In [7]:
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
tracks['track-name'] = tracks['track-name'].fillna('')

tracks = tracks.head(25000)

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(tracks['track-name'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


(10000, 9992)